# Multiprocessing with Pathos

In [3]:
from __future__ import print_function, unicode_literals

from collections import defaultdict
import pkg_resources
pkg_resources.require('SpaCy==2.1.0')
import spacy
import neuralcoref
import pandas as pd
from pathos.multiprocessing import Pool
import tqdm
import time

def apply_pipeline(text):
    text = nlp.pipe(text)
    return text

def load_processed_data(data="emails_processed.csv", nrows=5000):
    base_df = pd.read_csv(data, nrows=nrows)
    focused_df = base_df[['Message-ID', 'Subject']]
    return base_df, focused_df


def clean_threads(subject_df, subject_col='Subject'):
    subject_df = subject_df.astype(str)

    for index, row in subject_df.iterrows():
        row[subject_col] = row[subject_col].replace('Re:', '')
        row[subject_col] = row[subject_col].replace('re:', '')
        row[subject_col] = row[subject_col].replace('RE:', '')
        row[subject_col] = row[subject_col].replace('FW:', '')
        row[subject_col] = row[subject_col].strip()
    return subject_df


def get_threads(subject_df, base_df, subject_col='Subject', content_col='content'):
    subject_df = clean_threads(subject_df)

    # Group by subject
    subject_dict = subject_df.groupby(subject_col).groups

    # Get content for emails in same thread
    thread_dict = defaultdict(list)
    thread_dict_processed = defaultdict(list)
    for key, value in subject_dict.items():
        for value in subject_dict[key]:
            thread_dict[key].append(base_df.loc[value][content_col])

    for key in thread_dict.keys():
        thread_dict_processed[key] = ' '.join(thread_dict[key])

    return thread_dict_processed



In [4]:
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

base_df, focused_df = load_processed_data(nrows=1000)
threads = get_threads(focused_df, base_df)

In [ ]:
pool = Pool(5)
'''
for _ in tqdm(pool.imap_unordered(myfunc, range(100)), total=100):
    pass
'''
pbar = tqdm.tqdm(total=235)
def update(*a):
    pbar.update()
for i in range(pbar.total):
    pool.apply_async(apply_pipeline, [threads[key] for key in threads.keys()], callback=update)
pool.close()
pool.join()

  0%|          | 0/235 [00:00<?, ?it/s]

In [4]:
start = time.time()
threads_processed = []
for doc in nlp.pipe([threads[key] for key in threads.keys()], batch_size=1000, n_threads=5):
    threads_processed.append(doc)
print('Time elapsed: {} ms'.format((time.time()-start)*1000))

Time elapsed: 377758.2550048828 ms


In [ ]:
start = time.time()
docs_processed = []
for doc in nlp.pipe(base_df['content'].tolist(), batch_size=1000, n_threads=5):
    docs_processed.append(doc)
print('Time elapsed: {} sec'.format((time.time()-start)))

In [ ]:
# with mp.Pool(processes=2) as pool:
#     threads_processed = pool.map(apply_pipeline, [threads[key] for key in threads.keys()])
    
pool = Pool(processes=8)
for _ in tqdm.tqdm(pool.imap(nlp, [threads[key] for key in threads.keys()]), total=235):
    pass
pool.close()
pool.join()